# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
zeotis = Website("https://neuropathbhc.org/")#"https://www.zoetis.com/")
print(zeotis.links) # all links on the webpage
print(zeotis.get_contents())

['https://neuropathbhc.org/', 'https://www.facebook.com/neuropathbhch', 'https://www.instagram.com/neuropathbhc/', 'https://x.com/NeuropathH', 'https://www.linkedin.com/company/neuropath-behavioral/', 'https://neuropathbhc.org/', '#', '#', 'https://neuropathbhc.org/#book-consulting-services', 'https://neuropathbhc.org/', 'https://neuropathbhc.org/mission-and-vision/', 'https://neuropathbhc.org/areas-of-specialization/', 'https://neuropathbhc.org/wraparound-services/', 'https://neuropathbhc.org/approach/', 'https://neuropathbhc.org/faqs/', 'https://neuropathbhc.org/adult-services/#CounselingId', 'https://neuropathbhc.org/adult-services/#mental-health-counseling', 'https://neuropathbhc.org/adult-services/#telehealth-services', 'https://neuropathbhc.org/adult-services/#outpatient-counseling', 'https://neuropathbhc.org/adult-services/#crisis-stabilization', 'https://neuropathbhc.org/adult-services/#substance-abuse-recovery-treatment-options', 'https://neuropathbhc.org/adult-services/#upcom

Now if we're building a company brochure and we want to provide a web page and we want it to use that to gather more information, we want it to follow some of these links to figure out how it can. 
It can collect more information from them. But not all of these links are going to be relevant. Some of these links are going to be red herrings, like this thing here, which is probably from from 
one of the, uh, The analytics tags that's included. Or there are some other things here like that's going to be irrelevant. Now it's going to be really hard for us to write code to figure out whether or not a link is relevant 
for the purposes of generating a sales brochure. That's actually really hard. 

The other thing we might want to do is take a link to something like slash about and replace it with a full URL as well. And maybe that's easier to do with code, but it's still not a not a simple task by any means. 
The combined code to try and figure out which of these links are relevant and what's the full, full URL, including the the host.

That would be a lot of coding, but of course, it turns out that's the kind of thing that GPT four mini would be very good at doing for us. We could just ask it to do that as a task.
It's an example of taking a sort of nuanced, complicated task, and rather than trying to hand code it, we can just ship it off to a frontier model and say, do this for us, and that's what we're going
to do, and we're going to do that in the next video.

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

 so we're going to we're going to not only are we going to to use GPT four for this purpose, we're going to ask it to respond in the form of JSON in a way that specifies exactly the information that
we need back.

Now, later on, we're going to cover a technique called structured outputs, which is when we require the LLM to respond with a very specific format. We effectively we specify the format that it needs to respond in.
We're not going to do this today. We're just going to simply ask for JSON back. And we're going to tell it the format that it needs to use to reply. 

When you get more sophisticated, you you might need to use structured outputs. And in week eight, when we build our Agentic AI framework, we're going to do just that

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

#********************************
'''And when I said we're working with one shot prompting, that's really what I meant by giving it a specific

example that it could use with an about page and a careers page, and the way that we're specifying

the format is simply by giving it an example you can see we're asking for a dictionary.'''

"And when I said we're working with one shot prompting, that's really what I meant by giving it a specific\n\nexample that it could use with an about page and a careers page, and the way that we're specifying\n\nthe format is simply by giving it an example you can see we're asking for a dictionary."

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(zeotis))

Here is the list of links on the website of https://neuropathbhc.org/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://neuropathbhc.org/
https://www.facebook.com/neuropathbhch
https://www.instagram.com/neuropathbhc/
https://x.com/NeuropathH
https://www.linkedin.com/company/neuropath-behavioral/
https://neuropathbhc.org/
#
#
https://neuropathbhc.org/#book-consulting-services
https://neuropathbhc.org/
https://neuropathbhc.org/mission-and-vision/
https://neuropathbhc.org/areas-of-specialization/
https://neuropathbhc.org/wraparound-services/
https://neuropathbhc.org/approach/
https://neuropathbhc.org/faqs/
https://neuropathbhc.org/adult-services/#CounselingId
https://neuropathbhc.org/adult-services/#mental-health-counseling
https://neuropathbhc.org/adult-services/#telehealth-services
https://neuropathbhc.

In [9]:
# And now it's time to put all of this into a function which is going to call OpenAI.
def get_links(url):
    website = Website(url)
    # completion: your task is completing this conversation
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} # we tell OpenAI, we want Json object back in its response. OpenAI in its documentation recommend that it's still important that you mention in your prompt that a json response is required even if you specify format in this argument.
    )
    result = response.choices[0].message.content 
    # dot choices zero. So what's this about? Well, as it happens we can actually in the API request ask to have multiple variations if we want, 
    # if we wanted it to generate several possible variations of the response. And we haven't done that. So we're only going to get back one.
    # Uh, and so those variations come back in the form of these choices. But we've only got one. So choices zero is getting us the one and the only choice of the response back.
    return json.loads(result)  # we use json.loads to bring it back as JSON.

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://neuropathbhc.org/")#"https://huggingface.co")
huggingface.links

['https://neuropathbhc.org/',
 'https://www.facebook.com/neuropathbhch',
 'https://www.instagram.com/neuropathbhc/',
 'https://x.com/NeuropathH',
 'https://www.linkedin.com/company/neuropath-behavioral/',
 'https://neuropathbhc.org/',
 '#',
 '#',
 'https://neuropathbhc.org/#book-consulting-services',
 'https://neuropathbhc.org/',
 'https://neuropathbhc.org/mission-and-vision/',
 'https://neuropathbhc.org/areas-of-specialization/',
 'https://neuropathbhc.org/wraparound-services/',
 'https://neuropathbhc.org/approach/',
 'https://neuropathbhc.org/faqs/',
 'https://neuropathbhc.org/adult-services/#CounselingId',
 'https://neuropathbhc.org/adult-services/#mental-health-counseling',
 'https://neuropathbhc.org/adult-services/#telehealth-services',
 'https://neuropathbhc.org/adult-services/#outpatient-counseling',
 'https://neuropathbhc.org/adult-services/#crisis-stabilization',
 'https://neuropathbhc.org/adult-services/#substance-abuse-recovery-treatment-options',
 'https://neuropathbhc.org/

You'll also see that there are many of them that are not fully, uh, the full URL, including the thehost name.

And so you'll see that and that our call to GPT four mini has very well selected a subset of these fully

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

When we run this it's going to first go to OpenAI and collect those links and tell us it found these links. And then it's gathered all that information.
And so if we scroll down we'll see the result of all of these different web pages.


In [13]:
# print(get_all_details("https://huggingface.co"))

Well, we're going to call make a second call again to OpenAI. This time we've got something different to ask it. We're going to tell it. We've got all of this information. And we want you to use this to make a brochure.

And in some ways, you can think of this as being a bit like a mini agent solution, because we're making these two calls to GPT four, mini one to gather the relevant links and then one to build the brochure.

So it's a sort of toy version of Agentic. I obviously later will be doing things that are that are rather more substantive, but this certainly gives you a flavor for it.

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters, just in case
    return user_prompt

In [16]:
get_brochure_user_prompt("NeuroPath","https://neuropathbhc.org/") #"Zeotis", "https://www.zoetis.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://neuropathbhc.org/mission-and-vision/'}, {'type': 'careers page', 'url': 'https://neuropathbhc.org/careers/'}, {'type': 'internships page', 'url': 'https://neuropathbhc.org/internships/'}, {'type': 'services page', 'url': 'https://neuropathbhc.org/adult-services/'}, {'type': 'services for children page', 'url': 'https://neuropathbhc.org/services-for-children/'}, {'type': 'counseling programs page', 'url': 'https://neuropathbhc.org/counseling-programs/'}, {'type': 'referring organizations page', 'url': 'https://neuropathbhc.org/referring-organizations/'}, {'type': 'contact page', 'url': 'https://neuropathbhc.org/contact/'}]}


'You are looking at a company called: NeuroPath\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome - NEUROPATH BEHAVIORAL HEALTHCARE\nWebpage Contents:\nMon – Fri: 9:00 AM – 7:30 PM\ninfo@neuropathbhc.org\n888-337-6572\nAppointment\nHome\nAbout\nWho We Are\nNeuropath Behavioral Healthcare provides Comprehensive Therapeutic Support Services designed to address a\r\n            wide range of emotional, behavioral, and developmental challenges. Our approach is rooted in evidence-based\r\n            practices and our team of highly trained professionals is dedicated to providing individualized care for\r\n            every client.\nAbout\nMission & Vision\nAreas of Specialization\nWraparound Services\nOur Approach\nFAQs\nServices\nAdult Services\nConsultations\nMental Health Counseling\nTelehealth Services\nOutpatient Counseling\nCrisis Stabilization\nSubstance

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("NeuroPath","https://neuropathbhc.org/") #"Zeotis", "https://www.zoetis.com/") #We provided it and pulled it all together into this nice summary brochure.

Found links: {'links': [{'type': 'about page', 'url': 'https://neuropathbhc.org/mission-and-vision/'}, {'type': 'services page', 'url': 'https://neuropathbhc.org/areas-of-specialization/'}, {'type': 'services page', 'url': 'https://neuropathbhc.org/wraparound-services/'}, {'type': 'services page', 'url': 'https://neuropathbhc.org/approach/'}, {'type': 'adult services page', 'url': 'https://neuropathbhc.org/adult-services/'}, {'type': 'children services page', 'url': 'https://neuropathbhc.org/services-for-children/'}, {'type': 'referring organizations page', 'url': 'https://neuropathbhc.org/referring-organizations/'}, {'type': 'careers page', 'url': 'https://neuropathbhc.org/careers/'}, {'type': 'internships page', 'url': 'https://neuropathbhc.org/internships/'}, {'type': 'contact page', 'url': 'https://neuropathbhc.org/contact/'}]}


```markdown
# Welcome to NeuroPath Behavioral Healthcare

## About Us
At NeuroPath, we specialize in providing **Comprehensive Therapeutic Support Services** for individuals facing emotional, behavioral, and developmental challenges. Our mission is to offer personalized care through evidence-based practices, ensuring that each client receives tailored support from our highly trained professionals.

## Services Offered
### Adult Services
- **Mental Health Counseling**
- **Consultations and Outpatient Counseling**
- **Crisis Stabilization**
- **Substance Abuse Recovery Treatment Options**
  
### Children's Services
- **ABA Therapy for Autism and Spectrum Disorders**
- **In-Home and Telehealth Services**
- **Crisis Stabilization and Respite Care for Children with Developmental Disabilities**
  
### Telehealth & In-Person Support
We provide an array of services that are accessible either remotely or in-person to cater to your preferences and needs.

## Our Approach
We are committed to connecting clients with caring therapists who empower them using science-backed strategies for lasting change. We believe in a therapy process that:
1. **Encourages Connection:** Finding a therapist you trust.
2. **Tailors Experiences:** Customized plans that reflect your individual needs.
3. **Provides Insights:** Offering practical skills and real-life applications.

## Company Culture
At NeuroPath, we foster a **supportive and inclusive environment** for both our clients and team members. Our culture is built on empathy, collaboration, and a shared dedication to improving mental health. We encourage our employees to grow professionally and personally through ongoing training and open communication.

## Careers at NeuroPath 
Join our mission of creating effective therapeutic solutions! We are always on the lookout for passionate individuals to fill positions and internships across various specialties in behavioral health care. 

- **Current Opportunities:** Explore diverse roles tailored for both established professionals and emerging talent in behavioral healthcare.

### Contact Us
- Phone: **888-337-6572**
- Email: **info@neuropathbhc.org**
- Office Hours: **Mon – Fri 9:00 AM – 7:30 PM**

Take the first step towards effective therapy and a brighter future! [**Book a Session**](#).

## We Believe in Effective Therapy
Let us guide you toward a path forward—you're not alone.

---

For more details, please visit our website at [neuropathbhc.org](#).
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

and streaming is when the information flows back from the LLM and appears in what they call the typewriter style interface. And the way you do it is remarkably simple.

Uh, when you are creating your, uh, call to OpenAI chat completions, create, uh, if you want to not just receive it all back in one go, but you want it to flow back, you simply pass in another parameter stream equals true.

We'll find when we go and use Claude that it has a slightly different API, but this is one of the rare times when GPT and Claude are a bit different.

But that's how you do it with GPT stream equals true. Now what does that mean?
What it means is that what comes back into stream is no longer the the single text response, but instead you get back something that you can iterate over, and as you iterate over, each chunk of the response will come through, um, and you can just sort of say for chunk in stream print chunk. Now, in our case, we want to be a little bit smarter than that because we're we're showing this in

markdown.

In [29]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    # for chunk in stream: print(chunk.choices[0].delta.content or '', end='') # if we want to show the results once a time without streaming
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## About Us

**Hugging Face** is a leading platform in the AI community dedicated to building the future of machine learning. We focus on fostering collaboration among developers, researchers, and enterprises in the realm of artificial intelligence. Our mission is to facilitate accessibility to state-of-the-art models and datasets while achieving the evolution of AI technologies.

## Our Offerings

- **Models**: Explore a vast collection of over **400k+ models**, optimized for diverse tasks including natural language processing, image generation, and more.
- **Datasets**: Access and share more than **100k datasets** tailored for various machine learning applications.
- **Spaces**: Collaborate and run applications seamlessly, with a current host of **150k+ applications**.
- **Enterprise Solutions**: Provide organizations with advanced tools for building AI, including enterprise-grade security and dedicated support.

## Join the Community

Hugging Face is home to over **50,000 organizations** from various sectors, including notable companies like Meta, Amazon Web Services, and Microsoft, ensuring a robust ecosystem for innovation and implementation in AI.

## Company Culture

At Hugging Face, our culture is built on collaboration, innovation, and community engagement. We believe in:

- **Open Source**: Actively contributing to the foundational tools and technologies of machine learning.
- **Diversity**: Embracing diverse talents and ideas from all around the globe.
- **Learning**: Encouraging continuous growth through shared knowledge and experiences across our community.

## Careers at Hugging Face

We are always on the lookout for passionate individuals who are interested in AI and machine learning. If you are eager to innovate and be part of a thriving community, explore our career opportunities and see how you can contribute to our mission. **Join us** to help shape the future of AI!

### Current Job Opportunities

- Software Engineers
- Data Scientists
- Product Managers
- Community Advocates

## Connect with Us

For more information, resources, and to become part of our journey in the AI revolution, visit [Hugging Face](https://huggingface.co).

---

Together, let’s build the future of AI!

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>